# 📦 BNNR — Bring Your Own Data

This notebook shows how to use BNNR with **your own** classification or detection data.

Two paths:
1. **Classification** — Use `ImageFolder` structure + BNNR Python API (full control)
2. **Detection** — Use YOLO or COCO format + BNNR CLI (easiest) or Python API

⏱ **Runtime:** Depends on your data size. The examples below use small synthetic data for demonstration.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mateuszwalo/BNNR/blob/main/examples/bnnr_custom_data.ipynb)

## 0. Installation

In [ ]:
%pip install -q "bnnr[dashboard]"

## 1. Upload Your Data to Colab

### Option A: Google Drive (recommended for large datasets)
```python
from google.colab import drive
drive.mount('/content/drive')
# Your data is now at /content/drive/MyDrive/your_dataset/
```

### Option B: Direct upload (small datasets < 100 MB)
```python
from google.colab import files
uploaded = files.upload()  # Opens a file picker
# If it's a zip: !unzip uploaded_file.zip -d my_data/
```

### Option C: Download from URL
```python
!wget -q https://example.com/dataset.zip
!unzip -q dataset.zip -d my_data/
```

---

# Part A: Classification with Custom Data

## A1. Prepare Your Data

BNNR uses PyTorch's `ImageFolder` format. Organize your images like this:

```
my_data/
  train/
    class_a/
      img001.jpg
      img002.jpg
    class_b/
      img003.jpg
      ...
  val/
    class_a/
      img100.jpg
    class_b/
      img101.jpg
```

Each subfolder name = class name. Images can be any format (jpg, png, etc.).

**For this demo, we create a small synthetic dataset to show the workflow:**

In [ ]:
import os
import numpy as np
from PIL import Image

# Create synthetic ImageFolder data for demo
np.random.seed(42)
classes = ["cats", "dogs", "birds"]
for split in ["train", "val"]:
    n = 50 if split == "train" else 20
    for cls in classes:
        path = f"my_data/{split}/{cls}"
        os.makedirs(path, exist_ok=True)
        for i in range(n):
            img = np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8)
            Image.fromarray(img).save(f"{path}/{i:04d}.png")

print("Synthetic ImageFolder dataset created:")
for split in ["train", "val"]:
    for cls in classes:
        n = len(os.listdir(f"my_data/{split}/{cls}"))
        print(f"  {split}/{cls}: {n} images")

## A2. Load Data + Define Model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from bnnr import (
    BNNRConfig,
    BNNRTrainer,
    SimpleTorchAdapter,
    auto_select_augmentations,
)
from bnnr.icd import ICD, AICD
from bnnr.xai_cache import XAICache

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 3  # cats, dogs, birds


class IndexedDataset(Dataset):
    """Adds index to each sample for ICD/AICD caching."""
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        img, label = self.ds[idx]
        return img, label, idx


# IMPORTANT: NO Normalize()! BNNR augmentations need raw [0,1] tensors.
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder("my_data/train", transform=transform)
val_ds = datasets.ImageFolder("my_data/val", transform=transform)

train_loader = DataLoader(IndexedDataset(train_ds), batch_size=32, shuffle=True)
val_loader = DataLoader(IndexedDataset(val_ds), batch_size=32)

print(f"Train: {len(train_ds)}  Val: {len(val_ds)}  Classes: {train_ds.classes}")

# Simple model
class SimpleNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),  # target layer
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(nn.Flatten(), nn.Linear(128, num_classes))

    def forward(self, x):
        return self.classifier(self.features(x))

    @property
    def target_layer(self):
        return self.features[8]


model = SimpleNet(num_classes=NUM_CLASSES)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## A3. Configure, Start Dashboard, and Train

We use BNNR built-in augmentations + ICD/AICD to search for the best augmentation policy.

The next code cell also starts the dashboard before training:
- local URL for desktop view,
- embedded iframe on Colab,
- optional public/mobile URL when `NGROK_AUTHTOKEN` is set.



In [ ]:
adapter = SimpleTorchAdapter(
    model=model,
    criterion=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
    target_layers=[model.target_layer],
    device=DEVICE,
)

# Precompute XAI cache for ICD/AICD
xai_cache = XAICache(cache_dir="reports/custom_cls/xai_cache")
xai_cache.precompute_cache(
    model=model, train_loader=train_loader,
    target_layers=[model.target_layer],
    n_samples=len(train_ds), method="opticam",
)

# Build augmentations
augmentations = auto_select_augmentations()

# Add ICD and AICD
icd = ICD(
    model=model, target_layers=[model.target_layer],
    cache=xai_cache, probability=0.2, random_state=42,
)
icd.name = "icd"
augmentations.append(icd)

aicd = AICD(
    model=model, target_layers=[model.target_layer],
    cache=xai_cache, probability=0.2, random_state=43,
)
aicd.name = "aicd"
augmentations.append(aicd)

config = BNNRConfig(
    m_epochs=3,
    max_iterations=3,
    selection_metric="accuracy",
    device=DEVICE,
    xai_enabled=True,
    xai_method="opticam",
    report_dir="reports/custom_cls",
    event_log_enabled=True,
    seed=42,
)

print(f"{len(augmentations)} augmentation candidates")
print(f"Selection metric: {config.selection_metric}")



In [ ]:
# Live dashboard (works in Colab and local runs)

import os
import time as _time

try:
    from bnnr import start_dashboard
    dashboard_url = start_dashboard(config.report_dir, port=8080, auto_open=False, build_frontend=False)
    print(f"Dashboard server: {dashboard_url}")

    try:
        import google.colab  # noqa: F401
        from google.colab import output as colab_output

        _time.sleep(2)
        print("\n📊 Colab detected — embedding live dashboard below:")
        colab_output.serve_kernel_port_as_iframe(8080, height=900)

        # Optional: expose dashboard externally (phone/outside Colab session)
        # Requires: NGROK_AUTHTOKEN set in environment
        try:
            from pyngrok import ngrok

            token = os.environ.get("NGROK_AUTHTOKEN")
            if token:
                ngrok.set_auth_token(token)
                public_url = ngrok.connect(8080, "http").public_url
                print(f"🌍 Public dashboard URL: {public_url}")
            else:
                print("ℹ Optional public URL: install pyngrok and set NGROK_AUTHTOKEN.")
        except Exception as tunnel_exc:
            print(f"ℹ Tunnel not started: {tunnel_exc}")

    except ImportError:
        print("Open the URL above in your browser to view the dashboard.")

except Exception as dash_exc:
    print(f"Dashboard unavailable: {dash_exc}")



In [ ]:
trainer = BNNRTrainer(adapter, train_loader, val_loader, augmentations, config)
result = trainer.run()

print("\n" + "=" * 50)
print(f"Best path:    {result.best_path}")
print(f"Best metrics: {result.best_metrics}")
print(f"Selected:     {result.selected_augmentations}")
print("=" * 50)


---

# Part B: Detection with Custom Data

## B1. Via CLI (Easiest)

If you have data in YOLO or COCO format, the CLI is the simplest approach.

### YOLO format
Your dataset should have a `data.yaml` file:
```yaml
train: images/train
val: images/val
nc: 5
names: ['class_a', 'class_b', 'class_c', 'class_d', 'class_e']
```

In [ ]:
# YOLO format CLI command (uncomment and adjust paths):
# !python3 -m bnnr train \
#     -c examples/configs/detection/detection_underwater.yaml \
#     --dataset yolo \
#     --data-path /path/to/your/data.yaml \
#     --with-dashboard \
#     --batch-size 4 \
#     --epochs 3

# COCO format CLI command (uncomment and adjust paths):
# !python3 -m bnnr train \
#     -c examples/configs/detection/detection_underwater.yaml \
#     --dataset coco_mini \
#     --data-path /path/to/coco/dir \
#     --with-dashboard

print("Uncomment the appropriate command above and adjust the paths.")
print("\nSupported formats:")
print("  --dataset yolo      : YOLO format (data.yaml + images/ + labels/)")
print("  --dataset coco_mini : COCO format (annotations/ + images/)")

## B2. Via Python API (Full Control)

If you need full control, create a custom Dataset class. The key requirement:
each sample returns `(image_tensor, target_dict, index)`.

In [ ]:
from bnnr import BNNRConfig, BNNRTrainer
from bnnr.detection_adapter import DetectionAdapter
from bnnr.detection_augmentations import DetectionHorizontalFlip, DetectionVerticalFlip
from bnnr.detection_collate import detection_collate_fn_with_index
from bnnr.detection_icd import DetectionICD
from torch.utils.data import Dataset, DataLoader


class MyDetectionDataset(Dataset):
    """Example custom detection dataset.

    Replace the __getitem__ with your own data loading logic.
    """
    def __init__(self, num_samples=40, num_classes=3, seed=42):
        self.n = num_samples
        self.nc = num_classes
        self.rng = torch.Generator().manual_seed(seed)
        # Pre-generate data (replace with your actual data loading)
        self.data = []
        for _ in range(num_samples):
            img = torch.rand(3, 128, 128, generator=self.rng)
            n_obj = int(torch.randint(1, 4, (1,), generator=self.rng).item())
            boxes = []
            for _ in range(n_obj):
                x = int(torch.randint(0, 100, (1,), generator=self.rng).item())
                y = int(torch.randint(0, 100, (1,), generator=self.rng).item())
                boxes.append([x, y, x+20, y+20])
            labels = [int(torch.randint(0, num_classes, (1,), generator=self.rng).item())
                      for _ in range(n_obj)]
            self.data.append((img, {
                "boxes": torch.tensor(boxes, dtype=torch.float32),
                "labels": torch.tensor(labels, dtype=torch.int64),
            }))

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        img, target = self.data[idx]
        return img, target, idx  # Must return 3-tuple!


# Build everything
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn

det_model = fasterrcnn_mobilenet_v3_large_fpn(
    weights=None, weights_backbone=None, num_classes=4,
    rpn_pre_nms_top_n_train=200, rpn_post_nms_top_n_train=100,
    rpn_pre_nms_top_n_test=100, rpn_post_nms_top_n_test=50,
)

det_train = MyDetectionDataset(num_samples=32, seed=42)
det_val = MyDetectionDataset(num_samples=16, seed=99)

det_train_loader = DataLoader(det_train, batch_size=4, shuffle=True,
                              collate_fn=detection_collate_fn_with_index)
det_val_loader = DataLoader(det_val, batch_size=4,
                            collate_fn=detection_collate_fn_with_index)

det_adapter = DetectionAdapter(
    model=det_model,
    optimizer=torch.optim.SGD(det_model.parameters(), lr=0.005, momentum=0.9),
    device=DEVICE,
)

det_augs = [
    DetectionHorizontalFlip(probability=0.5, random_state=42),
    DetectionICD(probability=0.3, fill_strategy="gaussian_blur", random_state=42),
]

det_config = BNNRConfig(
    task="detection",
    m_epochs=2,
    max_iterations=2,
    device=DEVICE,
    xai_enabled=True,
    detection_xai_method="activation",
    report_dir="reports/custom_det",
    detection_class_names=["cat", "dog", "bird"],
    seed=42,
)

print("Training detection model on custom data...")
det_trainer = BNNRTrainer(det_adapter, det_train_loader, det_val_loader, det_augs, det_config)
det_result = det_trainer.run()

print("\n" + "=" * 50)
print(f"Best path:    {det_result.best_path}")
print(f"Best metrics: {det_result.best_metrics}")
print("=" * 50)

---

## Tips & Best Practices

### General
- **Do NOT use `transforms.Normalize()`** in the data pipeline. BNNR augmentations need raw `[0, 1]` tensors. Use `BatchNorm` in the model instead.
- **Always include sample index** in the dataset output (3-tuple) — required for ICD/AICD caching.
- **Start small**: Use `max_iterations=3`, `m_epochs=3` for initial testing, then scale up.

### Classification
- Include **ICD + AICD** in your candidate pool — they use XAI to intelligently augment images.
- For small datasets (<5K images), aggressive augmentations help more.
- Use `auto_select_augmentations()` as a starting point, then add ICD/AICD.

### Detection
- Use `detection_collate_fn_with_index` as the DataLoader's `collate_fn` — it handles variable-length targets.
- For detection, `num_classes` in the model = your classes + 1 (background).
- Boxes must be in **xyxy format** (x1, y1, x2, y2), not xywh.

### Dashboard
- Add `--with-dashboard` (CLI) or `event_log_enabled=True` (API) to enable monitoring.
- On Colab: embed dashboard with `google.colab.output.serve_kernel_port_as_iframe(8080)` after calling `start_dashboard()`; optional public/mobile URL can be added with `pyngrok`.
- Locally: the dashboard opens in your browser, or replay later with `bnnr dashboard serve --run-dir reports/your_run`

---

## Next Steps

- [Classification Demo](classification/bnnr_classification_demo.ipynb) — Full STL-10 demo with all augmentations
- [Detection Demo](detection/bnnr_detection_demo.ipynb) — Detection pipeline with XAI
- [Augmentations Guide](bnnr_augmentations_guide.ipynb) — Visual guide to every BNNR augmentation
- [Configuration Reference](../docs/configuration.md) — All BNNRConfig fields explained